In [ ]:
using Random
using Plots
using Distributions
using DifferentialEquations
using Statistics
Evolutionary_Complexity = []
 a::Int32 = 1.0
 v::Float32 = 6*10^-4
 M::Int32 = 50 #This is the amount of genes we want in our network
Orbit_Plotting = true  #Currently not used yet
Epi_Plotting = true #Currently not used yet
Counter = 0 #Used to randomise the mutations later

SplitTime::Float32 = 5 #Time before a splitting event happens
KArray = zeros(Float32,1,M) #Diffusion/Mean Field array is set to zero everywhere
KArray[1:10] .= 0.2 #Here we can manually adjust which genes will have diffusion

function Setting_Up_Randomly(M,Sparse,rng=MersenneTwister(123))
    "Function used to randomly with seed define our c_i's, J_ij's and the initial positions
    Input:
    M   = Amount of Genes in our system
    rng = Random Julia seed to make sure we can recreate the same initial conditions
    Output:
    CArray          = C_i's used, can only be between -0.5 and 0.5 now
    PositionArray   = Random Initial position, between -1 and 1 for all genes, epifactors = 0
    ParameterMatrix = J_ij's Currently a dense network with values between -0.5 and 0.5
    "
    ParameterMatrix = zeros(M,M)
    CArray = zeros(1,M)
    PositionArray = zeros(1,2*M)
    
    CArray = rand(rng,Float32,M).-0.5; #Random c_j between -0,5 and 0.5
    PositionArray[1:M] = 2*(rand(rng,Float32,M).-0.5); #Random Initial position 
    ParameterMatrix = rand(rng,Float32,M,M).-0.5; #J_jk's
    if Sparse
       for i=1:M
            for j=1:M
                if rand() < 5/M #Average of 5 connections per gene
                   ParameterMatrix[i,j] = 0 
                end
            end
        end
    end
    return CArray, PositionArray, ParameterMatrix
end

function Epigenetic_Evolution(du,u,Parameters,Time)
    "Function used to define the ODE, needs to differentiate the boundaries and splits for diffusion.
    Input:
    du         = Used by ODESolver to calculate step, don't need to give input
    u          = These are our x's for the dx = f(x) solving. Basically the parameters that change
    Parameters = These are constants we have to give for our solver, it's a tuple consisting of J_ij's, C_i's, Diffusion, Splits
    Time       = This determines for how long the ODE will run
    Output is done through ODESolver, not this function
    "
    ParameterMatrix = Parameters[1]
    CArray = Parameters[2]
    KArray = Parameters[3]
    Splits = Parameters[4]
    if Splits == 0  #Cannot difuse
        for j=1:M*2^Splits #Gene evolution
            du[j] = tanh(40*(sum(ParameterMatrix[j,:].*u[1:M]/sqrt(M)) + u[j+M] + CArray[j])) - u[j] 
        end
    elseif Splits == 1   #Only diffusion with 1 neighbour, 2 cells in total
        #i=1 is the first cell
        for j=1:M
           du[j] = tanh(40*(sum(ParameterMatrix[j,:].*u[1:M]/sqrt(M)) + u[j+M*2^Splits] + CArray[j])) - u[j] + KArray[j]*(u[j+M] - u[j])
        end
        #i=2 is the second cell
        for j=1:M
           du[j+M] = tanh(40*(sum(ParameterMatrix[j,:].*u[M+1:2*M]/sqrt(M)) + u[j+M+M*2^Splits] + CArray[j])) - u[j+M] + KArray[j]*(u[j] - u[j+M])
        end
        else #Now we have 4 or more cells
        for i=0:2^Splits-1 #Loop over all cells, Can defuse but watch for boundaries (next if statement)
            if i==0 #Boundaries for diff.
                for j=1:M
                    du[j] = tanh(40*(sum(ParameterMatrix[j,:].*u[1:M]/sqrt(M)) + u[j+M*2^Splits] + CArray[j])) - u[j] + KArray[j]*(u[j+M] - u[j])
                end
        elseif i == 2^Splits-1 #Boundaries for diff.
                for j=1:M
                    du[j+M*i] = tanh(40*(sum(ParameterMatrix[j,:].*u[i*M+1:M*2^Splits]/sqrt(M)) + u[j+M*i+M*2^Splits] + CArray[j])) - u[j+M*i] + KArray[j]*(u[j+M*(i-1)] - u[j+M*i])
                end
            else # Free to diffuse in both directions
                for j=1:M
                    du[j+M*i] = tanh(40*(sum(ParameterMatrix[j,:].*u[M*i+1:M*(i+1)]/sqrt(M)) + u[j+M*i+M*2^Splits] + CArray[j])) - u[j+M*i] + KArray[j]*(u[j+M*(i-1)] - 2u[j+M*i] + u[j+M*(i+1)])
                end
            end                
    end
end
    
    for j=1:M*2^Splits #Epigenetic Evolution, no need to worry about boundaries here so simple loop
        du[M*2^Splits + j] = v*(a*u[j] - u[j+M*2^Splits])
        
    end
    
end





function Noisy_Splitting(solution,Total_Splits,M,Seed)
    " Once our cell decides to split, we decide to introduce some random noise in how these genes get distributed.
    This function takes the cells, doubles them and then gives the daughters cell the same gene parameters +- noise
    one daughter gets + the other gets -. For the epigeneitc factors they are simply inherited and do not see any noise
    Input:
    solution     = These are the initial conditions of the parent cells that we need to split 
    (First genes of all cells, then all epigenetic factors)
    Total_Splits = Amount of times the cell has split (including this current split)
    M            = Amount of Genes
    Seed         = Random Noise seed for reproducing results
    Output:
    SplittedArray = New array of the genes and epigenetic factors of the daughter cells
    How is this structured? Suppose we take M=3 for convenience, we would have [g1,g2,g3,e1,e2,e3] for the first parent cell
    The daughters become: [g11,g12,g13,g21,g22,g23,e11,e12,e13,e21,e22,e23]
    Where we have genes of daughter 1, then genes of daughter 2, then epi of daughter 1 and epi of daughter 2.
    When the Daughters split we get the genes of Children of cell 1, then genes of Children of cell 2 and same for epi.
    "
    Random.seed!(Seed)
    s = Total_Splits #For shortening
    
    SplittedArray = zeros(Float32,1,2^(s+1)*M) #2^(s+1), 2M is the normal length, 4M after 1 split, 8M after 2 splits etc.
    for g=1:2^(s-1)*M #looping over the length of the parent
        Gaussian = Normal{Float32}(0.0f0,0.01f0)
        Displace = rand(Gaussian) #Noise for split
        index = Int32.(floor((g-1)/M)*M +g) #example M=10: Index 1-> 1 and 11, index 2->2 and 12 ... 
        #index 10->10 and 20, index 11->21 and 30 and so on. 
        
        #Genes
        SplittedArray[index] = solution[g] + Displace
        SplittedArray[index+M] = solution[g] - Displace
        
        #Epigenetic
        SplittedArray[index + 2^(s)*M] = solution[g + 2^(s-1)*M]
        SplittedArray[index+M + 2^(s)*M] = solution[g + 2^(s-1)*M]   
    end
    return SplittedArray  
end

function Differentiaton_Detection_Parameter(M,Epigenetic_Plot)
    
    b = Int[]
    Counter = Int[]
    Epi_Bitwise = signbit.(Epigenetic_Plot)
    
    for i=1:length(Epi_Bitwise[:,1])
        Bit_Represent = 0
        for j=1:M
            Bit_Represent += Epi_Bitwise[i,j]*2^(j-1)
        end
        #print(Bit_Represent)
        #print("     ")
        if !(Bit_Represent in b)
            push!(b,Bit_Represent)
            push!(Counter,1)
        else 
            Counter[findfirst(==(Bit_Represent), b)] += 1
        end
        
    end
    
    #print(b)
    Counter = sort(Counter,rev=true)
    #print(Counter)
    Score = 0
    #Make here the formula to turn b and the counter into a score
    for i=1:length(b)
       Score += (i-1)*Counter[i] 
    end
    
    return length(b)
end


function Mutate_Parameters(ParameterMatrix,M,Mutations,Seed)
    
    Random.seed!(Seed)
    for j=1:Mutations #Risk of remutating the same connection is roughly (Mutations-1)/M² (2% for M=10, Mutations = 3)
        #Could write something to prevent the network from being overly dense
        i = rand(1:M)
        j = rand(1:M)
        looping = true
        if ParameterMatrix[i,j] == 0
           ParameterMatrix[i,j] = Float32(rand().-0.5)
            while looping
                k = rand(1:M)
                l = rand(1:M)
                if ParameterMatrix[k,l] != 0
                    looping = false
                    ParameterMatrix[k,l] = 0
                end
            end
        else
            while looping
                Added = Float32(0.2*rand().-0.1)
                if abs(ParameterMatrix[i,j]+Added) < 1
                    ParameterMatrix[i,j] += Added
                    looping = false
                end
            end
        end
        
    end
    return ParameterMatrix
end


In [ ]:
MultiplyBroad_(a,b) = a .* b

SummingDim2_(a) = sum(a,dims=2) 

HyperTanh_(a) = tanh.(40*a)

function Epigenetic_Evolution3(du,u,Parameters,Time)
    "Function used to define the ODE, needs to differentiate the boundaries and splits for diffusion.
    Input:
    du         = Used by ODESolver to calculate step, don't need to give input
    u          = These are our x's for the dx = f(x) solving. Basically the parameters that change
    Parameters = These are constants we have to give for our solver, it's a tuple consisting of J_ij's, C_i's, Diffusion, Splits
    Time       = This determines for how long the ODE will run
    Output is done through ODESolver, not this function
    "
    ParameterMatrix = Parameters[1]
    CArray = Parameters[2]
    KArray = Parameters[3]
    Splits = Parameters[4]
    
    #⊗_(Wi, bi) = Wi .* bi
    
    for i::Int32 =0:2^Splits-1 #Loop over all cells, Can defuse but watch for boundaries (next if statement)
        #du[M*i+1:M + M*i] = HyperTanh_(SummingDim2_(MultiplyBroad_(ParameterMatrix[1:M,:],u[M*i+1:M*(i+1)]'))
        #    .+ u[M*i+M*2^Splits + 1:M+M*i+M*2^Splits] .+ CArray[1:M]) .- u[M*i+1:M+M*i]
        
        
        
        du[M*i+1:M + M*i] = tanh.(40*(sum(MultiplyBroad_(ParameterMatrix[1:M,:],u[M*i+1:M*(i+1)]'),dims=2)
                /Float32(sqrt(M)) .+ u[M*i+M*2^Splits + 1:M+M*i+M*2^Splits] .+ CArray[1:M])) .- u[M*i+1:M+M*i]
        
    end
    
    
    
    #Epigenetic Evolution, no need to worry about boundaries here so simple loop
    du[M*2^Splits + 1:M*2^Splits + M*2^Splits] = v*(a*u[1:M*2^Splits] .- u[M*2^Splits+1:M*2^Splits+M*2^Splits])
        
    
    
end

function Epigenetic_Evolution2(du,u,Parameters,Time)
    global PlotSecond
    "Function used to define the ODE, needs to differentiate the boundaries and splits for diffusion.
    Input:
    du         = Used by ODESolver to calculate step, don't need to give input
    u          = These are our x's for the dx = f(x) solving. Basically the parameters that change
    Parameters = These are constants we have to give for our solver, it's a tuple consisting of J_ij's, C_i's, Diffusion, Splits
    Time       = This determines for how long the ODE will run
    Output is done through ODESolver, not this function
    "
    ParameterMatrix = Parameters[1]
    CArray = Parameters[2]
    KArray = Parameters[3]
    Splits = Parameters[4]
    Cells = 2^Splits
    
    
    for i=0:Cells-1 #Loop over all cells, Can defuse but watch for boundaries (next if statement)
        for j=1:M
            
            #du[j + M*i] = tanh(40*(sum(MultiplyBroad_(ParameterMatrix[j,:],u[M*i+1:M*(i+1)]))/sqrt(M) 
            #            + u[j+M*i+M*2^Splits] + CArray[j])) - u[j+M*i]
            du[j + M*i] = tanh(40*(sum(MultiplyBroad_(ParameterMatrix[j,:],u[M*i+1:M*(i+1)]))/sqrt(M) 
                    + u[j+M*i+M*2^Splits] + CArray[j])) - u[j+M*i] + KArray[j]*(sum(u[j:M:j+M*(Cells-1)])/Cells - u[j + M*i])
        end   
       
    end
    
    
    for j=1:M*2^Splits #Epigenetic Evolution, no need to worry about boundaries here so simple loop
        du[M*2^Splits + j] = v*(a*u[j] - u[j+M*2^Splits])
        
    end
    
end

In [ ]:
Mean_Field = true
N::Int32 = 128
K::Int32 = N/4
Splits::Int32 = 6 #How often will we split
CArray_Collection = zeros(Float32,N,M)
PositionArray_Collection = zeros(Float32,N,2*M)
ParameterMatrix_Collection = zeros(Float32,N,M,M)
Final_Epigenetic_Factors = zeros(Float32,N,2^Splits,M)
Sparse = true
for i=1:N
    RandomInitial = 1000*i+5 #Seed for Initial conditions

    CArray_Collection[i,:], PositionArray_Collection[i,:], ParameterMatrix_Collection[i,:,:] = Setting_Up_Randomly(M,Sparse,MersenneTwister(RandomInitial))
    #Sets the initial conditions that we will be reusing in the simulation (Change RandomInitial for different values)
    
end


Random_Noise = 574 #seed for splitting noise

#Need to define outside for loop, otherwise it will be destroyed afterwards
Times = 0
Genes = 0


In [ ]:
#abcde = [0.9999978242832007, 0.9993739729292428, -0.999999670756514, 0.9999998724336769, 0.5539552247653006, 
#    0.9998685418731212, 0.9999999824504882, -0.9999992915887304, -0.9999946589034421, -0.9999306923553437, 
#    0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
#bcdef =[0.9999978242832007, 0.9993739729292428, -0.999999670756514, 0.9999998724336769, 0.5539552247653006, 
#    0.9998685418731212, 0.9999999824504882, -0.9999992915887304, -0.9999946589034421, -0.9999306923553437, 
#    0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
#print(abcde-bcdef)
#tester27 = [0.41951120694991145 0.02206833265691764 -0.2522165714408503 -0.12258934295305923 0.14451525916321478 0.25984627759206924 0.09118927018808795 0.0018633313680054444 -0.12993445825973599 0.16750530240761297; 0.2270377213832072 -0.4877028176562048 -0.30989153593070967 0.2842586971402873 -0.382187480132093 0.1098871176739855 0.023172125232848288 -0.014269384822029817 -0.39349757008797714 -0.16991985331217926; 0.056957854210280746 0.17745010555976537 0.06309437895001749 0.15823651126077123 -0.12826829819860475 0.10563537784854714 -0.12142588686930379 0.0817905231144618 -0.01928816787890007 0.01691191452163016; 0.029297421574967897 -0.026312271144937897 0.1944300596611509 0.23435854945609888 0.05276502189107581 0.14827760027774464 0.09442285266387866 -0.29257710808819726 -0.29234138728574915 -0.20756522227910032; 0.20071416198565015 -0.04584553879743179 0.28270040372460875 -0.35341341945104127 -0.4052523576920079 -0.4019216851100208 -0.17451980197315575 0.334321854405714 -0.2976019431014126 -0.02114248225992666; 0.019017891813995647 -0.05393441516760104 0.01083098213390193 0.05646509681541449 -0.08844131244572472 0.03185029364298997 -0.06028248193731387 0.010796815145555477 0.04720472596071227 0.06515103146263748; 0.002778597743601008 0.024511186094990985 0.0321702011331745 0.15349792193384781 -0.019957779379534114 -0.14711931571476694 0.04310755337937733 0.18649490294210969 -0.13667926852237997 -0.19993504031178708; -0.06874028163537281 0.02497258455862932 -0.10560247752315562 -0.028497348610488908 -0.050347361903518226 -0.01892258914762258 0.10732307991738661 -0.08452776861644196 0.0754691391500903 0.03734342508890715; 0.12617841748928796 0.04128689256047466 -0.198412605922361 0.212752409743899 0.08216860418225594 -0.3169698462885385 -0.2890465782149878 0.20932120084917644 -0.3004401271377864 -0.34774039351077224; 0.20483539495079806 -0.006108085264768492 -0.2848146075742378 -0.07597472916316465 -0.2450311355727375 -0.2138886558217775 0.05712821871215548 -0.2082781007592467 -0.09366627057850488 0.04548722635552845]
#tanh.(40*sum(tester27,dims=2))
#z = [0.9999995106928758, -0.9999999999988213, 0.9998990508623163, -0.6779359765280739, -0.9999999995916484, 0.45342020459711807, -0.6488172055268311, -0.8876533421188644, -0.9999999947355285, -0.999999998057433, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
#abcde-z

In [ ]:
#tweede = [0.41951120694991145, -0.025858055217654344, -0.1060053497997806, 0.0884113171571641, 0.1478999360212326, 0.0583727479980522, -0.05028825015162362, 0.0005427692292361335, -0.10963538414645455, 0.11982629221702314]
#eerste = [0.41951120694991145 -0.025858055217654344 -0.1060053497997806 0.0884113171571641 0.1478999360212326 0.0583727479980522 -0.05028825015162362 0.0005427692292361335 -0.10963538414645455 0.11982629221702314; -0.19376337156757387 -0.4877028176562048 0.11115718593675596 0.17496153003629203 0.3338138868047123 -0.02106755248765236 0.010905920169905511 0.0035473499741127865 0.2833623338406405 0.10373883099568351; 0.13551877082315464 -0.49470742983966465 0.06309437895001749 -0.2715236586143086 -0.3123341322799182 0.05646103870569989 0.1593235235135666 0.056685756069587846 -0.03872248446595094 0.028784730723535615; -0.040623211785314604 -0.042749351316889846 -0.11330848472657856 0.23435854945609888 -0.0748764523473356 -0.04618639278034551 0.07220125685135415 0.11817084008837586 0.34202792270598875 0.2058841943920198; 0.19612083627216406 0.052489101385224994 0.11609842133204963 0.24904848225755602 -0.4052523576920079 -0.08822279196996771 0.09404012885241268 0.09515587573441804 -0.24536232880403394 -0.01477832648042007; 0.08465814211248633 0.28131874500720416 0.020264148807983058 -0.18127653084011963 -0.40291721150276055 0.03185029364298997 0.14798587665159368 0.013999971451697443 0.17730388189115856 0.2074680468870669; -0.0050385189307897045 0.05207962876585971 -0.024518006614553497 0.20074043443330208 0.03703767474209351 0.059929485791980174 0.04310755337937733 -0.09850750198760166 0.2091251494900693 0.2593520543457898; -0.23598596994339663 -0.10045341499097082 -0.15237129179681227 0.07055608505789243 -0.1768910565543938 -0.014593150979418901 -0.20318459984053247 -0.08452776861644196 0.2186103277139709 0.09170913279545405; 0.14954044671053052 -0.05733398535659602 -0.09883187262133641 -0.1818457806627159 0.09966296124496267 -0.08438887277744182 0.18891403054895353 0.07226232630061112 -0.3004401271377864 -0.29481684927273927; 0.2863396182940714 0.010004787428643103 -0.16733734090028365 0.07659505671581537 -0.3505516300401174 -0.0671672903564339 -0.04404026310864198 -0.08480963036375287 -0.11048061150508856 0.04548722635552845]
#tanh(40*(sum(tweede)/sqrt(M)))
#abcde = [0.9999978242832007, 0.9993739729292428, -0.999999670756514, 0.9999998724336769, 0.5539552247653006, 
#    0.9998685418731212, 0.9999999824504882, -0.9999992915887304, -0.9999946589034421, -0.9999306923553437, 
#    0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
#tanh.(40*(sum(eerste,dims=2)/sqrt(M)))

In [ ]:
using ProgressMeter
using TickTock
tick()
blabla = 2
Orbit1 = 0
Orbit2 = 0
Orbit3 = 0
Orbit4 = 0
Orbit5 = 0
Orbit6 = 0
EpiOrbit1 = 0
EpiOrbit2 = 0
EpiOrbit3 = 0
EpiOrbit4 = 0
EpiOrbit5 = 0
EpiOrbit6 = 0
FinalOrbit = 0
FinalEpiOrbit = []
@showprogress 1 "Networks" for x=1:N
    CArray = CArray_Collection[x,:]
    ParameterMatrix = ParameterMatrix_Collection[x,:,:]
    PositionArray = PositionArray_Collection[x,:]
    for s=1:Splits #Looping over the splits
        Parameters = (ParameterMatrix, CArray,KArray,Int32(s-1)) #We have to update the splits every rerun, rest stays same
        tspan = (Float32(0.0),SplitTime) #How long ODE will run
        if Mean_Field
            prob = ODEProblem(Epigenetic_Evolution2,PositionArray,tspan,Parameters) #ODEdefining
        else
            prob = ODEProblem(Epigenetic_Evolution3,PositionArray,tspan,Parameters) #ODEdefining
        end
        #sol = solve(prob) #Solving
        #sol = solve(prob,Tsit5())
        sol = solve(prob,BS3())
        #sol = solve(prob,alg_hints = [:nonstiff]) #Solving
        Times = sol.t #Save the time, doesn't get used yet though
        StepAmount = length(Times)  #Amount of data points per gene
        Genes = sol 
        PositionArray = Genes[StepAmount] #Saving the genes and epi factors
        #Now we must take all the values from the updated PositionArray and give appropriate terms to the daughter cells
        PositionArray = Noisy_Splitting(PositionArray,s,M,Random_Noise)  #Splitting function
        if x==blabla
            if s==1
                Orbit1 = [Times,Genes[1:M,1:StepAmount]]
                EpiOrbit1 = [Times,Genes[M+1:2*M,1:StepAmount]]
            elseif s==2
                Orbit2 = [Times.+SplitTime,Genes[1,1:M*2,1:StepAmount]]
                EpiOrbit2 = [Times,Genes[1,2*M+1:2^2*M,1:StepAmount]]
            elseif s==3
                Orbit3 = [Times.+SplitTime*2,Genes[1,1:M*2^2,1:StepAmount]]
                 EpiOrbit3 = [Times,Genes[1,M*2^2+1:2^3*M,1:StepAmount]]

            elseif s==4
                Orbit4 = [Times.+SplitTime*3,Genes[1,1:M*2^3,1:StepAmount]]
                EpiOrbit4 = [Times,Genes[1,M*2^3+1:2^4*M,1:StepAmount]]

            elseif s==5
                Orbit5 = [Times.+SplitTime*4,Genes[1,1:M*2^4,1:StepAmount,]]
                EpiOrbit5 = [Times,Genes[1,M*2^4+1:2^5*M,1:StepAmount]]
            elseif s==6
                Orbit6 = [Times.+SplitTime*5,Genes[1,1:M*2^5,1:StepAmount,]]
                EpiOrbit6 = [Times,Genes[1,M*2^5+1:2^6*M,1:StepAmount]]


            else
                print("Error, add more orbittracker")
                break
            end
        end

        #Now we rerun and keep splitting
    end

    Final_Run_Time::Int32 = 5000 #Long evolving of the cells after splitting to get the final epigenetic state
    Parameters = (ParameterMatrix, CArray,KArray,Int32(Splits)) 
    tspan = (Int32(0.0),Final_Run_Time)

    prob = ODEProblem(Epigenetic_Evolution3,PositionArray,tspan,Parameters) #ODEdefining
    #sol = solve(prob,Tsit5())
    sol = solve(prob,BS3())
    #sol = solve(prob,alg_hints = [:nonstiff]) #Solving
    #sol = solve(prob)
    Times = sol.t
    StepAmount = length(Times)
    GenesEpi = sol
    PositionArray = GenesEpi[StepAmount]
    Genes = PositionArray[1:2^Splits*M] #Saves all the final genes
    Epigenetic = PositionArray[1+2^Splits*M:2^(Splits+1)*M]; #Saves all the final Epigenetic factors
    if x==blabla
        FinalOrbit = [Times,GenesEpi[1,1:M*2^(Splits-1),1:StepAmount]]
        FinalEpiOrbit = GenesEpi[1,M*2^(Splits-1)+1:M*2^Splits,1:StepAmount]
    end
    Epigenetic_Plot = zeros(Int.(length(Epigenetic)/M),M)
    for l=1:M
        
        Epigenetic_Plot[:,l] = Epigenetic[l:M:end]  #Reshapes the factors to plot easily    
    end
    
    Final_Epigenetic_Factors[x,:,:] = Epigenetic_Plot
    
end
print("Jobs Done")
tock()

In [ ]:
if Orbit_Plotting
    GeneTimes = cat(Orbit1[1],Orbit2[1],Orbit3[1],Orbit4[1],Orbit5[1],Orbit6[1],dims=1)
    GeneEvolution = zeros(M*2^(Splits),length(GeneTimes),2)
    Length = 0
    for s=1:6  
        if s==1
            for i=1:M
                Length = length(Orbit1[2][i,:])
                for j=1:2^(Splits)  
                    GeneEvolution[i+M*(j-1),1:Length,1] = Orbit1[2][i,:]
                    GeneEvolution[i+M*(j-1),1:Length,2] = EpiOrbit1[2][i,:]
                end
            end

        elseif s==2
            for i=1:2
                for j=1:2^(Splits-s+1)
                    GeneEvolution[2^(Splits-s+1)*M*(i-1)+ (j-1)*M+1:M*j+2^(Splits-s+1)*M*(i-1),Length+1:Length + length(Orbit2[2][1,:]),1] = Orbit2[2][M*(i-1)+1:M*i,:]
                    GeneEvolution[2^(Splits-s+1)*M*(i-1)+ (j-1)*M+1:M*j+2^(Splits-s+1)*M*(i-1),Length+1:Length + length(Orbit2[2][1,:]),2] = EpiOrbit2[2][M*(i-1)+1:M*i,:]

                end
            end
            Length = Length + length(Orbit2[2][1,:])
        elseif s==3
            for i=1:4
                for j=1:2^(Splits-s+1)
                    GeneEvolution[2^(Splits-s+1)*M*(i-1)+ (j-1)*M+1:M*j+2^(Splits-s+1)*M*(i-1),Length+1:Length + length(Orbit3[2][1,:]),1] = Orbit3[2][M*(i-1)+1:M*i,:]
                    GeneEvolution[2^(Splits-s+1)*M*(i-1)+ (j-1)*M+1:M*j+2^(Splits-s+1)*M*(i-1),Length+1:Length + length(Orbit3[2][1,:]),2] = EpiOrbit3[2][M*(i-1)+1:M*i,:]
                end
            end
            Length = Length + length(Orbit3[2][1,:])

        elseif s==4
            for i=1:8
                for j=1:2^(Splits-s+1)
                    GeneEvolution[2^(Splits-s+1)*M*(i-1)+ (j-1)*M+1:M*j+2^(Splits-s+1)*M*(i-1),Length+1:Length + length(Orbit4[2][1,:]),1] = Orbit4[2][M*(i-1)+1:M*i,:]
                    GeneEvolution[2^(Splits-s+1)*M*(i-1)+ (j-1)*M+1:M*j+2^(Splits-s+1)*M*(i-1),Length+1:Length + length(Orbit4[2][1,:]),2] = EpiOrbit4[2][M*(i-1)+1:M*i,:]

                end
            end
            Length = Length + length(Orbit4[2][1,:])
        elseif s==5
            for i=1:16
                for j=1:2^(Splits-s+1)
                    GeneEvolution[2^(Splits-s+1)*M*(i-1)+ (j-1)*M+1:M*j+2^(Splits-s+1)*M*(i-1),Length+1:Length + length(Orbit5[2][1,:]),1] = Orbit5[2][M*(i-1)+1:M*i,:]
                    GeneEvolution[2^(Splits-s+1)*M*(i-1)+ (j-1)*M+1:M*j+2^(Splits-s+1)*M*(i-1),Length+1:Length + length(Orbit5[2][1,:]),2] = EpiOrbit5[2][M*(i-1)+1:M*i,:]

                end
            end
            Length = Length + length(Orbit5[2][1,:])
        elseif s==6
            for i=1:32
                for j=1:2^(Splits-s+1)
                    GeneEvolution[2^(Splits-s+1)*M*(i-1)+ (j-1)*M+1:M*j+2^(Splits-s+1)*M*(i-1),Length+1:Length + length(Orbit6[2][1,:]),1] = Orbit6[2][M*(i-1)+1:M*i,:]
                    GeneEvolution[2^(Splits-s+1)*M*(i-1)+ (j-1)*M+1:M*j+2^(Splits-s+1)*M*(i-1),Length+1:Length + length(Orbit6[2][1,:]),2] = EpiOrbit6[2][M*(i-1)+1:M*i,:]
                end
            end
        end
    end
    j=1
    pa = plot(GeneTimes,GeneEvolution[j,:,1]) 
    for i=1:2^(Splits-1)-1
      pa = plot!(GeneTimes,GeneEvolution[j+M*i,:,1]) 
    end
    ylims!(-1, 1)
    j=2
    pb = plot(GeneTimes,GeneEvolution[j,:,1]) 
    for i=1:2^(Splits-1)-1
      pb = plot!(GeneTimes,GeneEvolution[j+M*i,:,1]) 
    end
    ylims!(-1, 1)
    j=3
    pc = plot(GeneTimes,GeneEvolution[j,:,1]) 
    for i=1:2^(Splits-1)-1
      pc = plot!(GeneTimes,GeneEvolution[j+M*i,:,1]) 
    end
    ylims!(-1, 1)
    j=4
    pd = plot(GeneTimes,GeneEvolution[j,:,1]) 
    for i=1:2^(Splits-1)-1
      pd = plot!(GeneTimes,GeneEvolution[j+M*i,:,1]) 
    end
    ylims!(-1, 1)
    k = plot(pa,pb,pc,pd,layout=(4,1),legend=false)
    
end

In [ ]:
j=1
pa = plot(GeneTimes,GeneEvolution[j,:,1]) 
for i=1:2^(Splits)-1
  pa = plot!(GeneTimes,GeneEvolution[j+M*i,:,1]) 
end
ylims!(-1, 1)
j=2
pb = plot(GeneTimes,GeneEvolution[j,:,1]) 
for i=1:2^(Splits)-1
  pb = plot!(GeneTimes,GeneEvolution[j+M*i,:,1]) 
end
ylims!(-1, 1)
j=3
pc = plot(GeneTimes,GeneEvolution[j,:,1]) 
for i=1:2^(Splits)-1
  pc = plot!(GeneTimes,GeneEvolution[j+M*i,:,1]) 
end
ylims!(-1, 1)
j=4
pd = plot(GeneTimes,GeneEvolution[j,:,1]) 
for i=1:2^(Splits)-1
  pd = plot!(GeneTimes,GeneEvolution[j+M*i,:,1]) 
end
ylims!(-1, 1)
k = plot(pa,pb,pc,pd,layout=(4,1),legend=false)

for j=5:6
    pe = plot(GeneTimes,GeneEvolution[j,:,1]) 
    for i=1:2^(Splits)-1
      pe = plot!(GeneTimes,GeneEvolution[j+M*i,:,1]) 
    end
    ylims!(-1, 1)
    
    display(pe)
end
k = plot(pa,pb,pc,pd,layout=(4,1),legend=false) 

In [ ]:
j = 14
pe = plot(GeneTimes,GeneEvolution[j,:,1]) 
for i=1:2^(Splits)-1
  pe = plot!(GeneTimes,GeneEvolution[j+M*i,:,1],legend=false) 
end

ylims!(-1, 1)

display(pe)

In [ ]:
j = 14
i=0
pe = plot(GeneTimes,GeneEvolution[j+i*M,:,1]) 
ylims!(-1, 1)

In [ ]:
i=38
#plot(Orbit3[1],Orbit3[2][i,:])
#plot!(Orbit3[1],Orbit3[2][50+i,:])
#plot!(Orbit3[1],Orbit3[2][100+i,:])
#plot!(Orbit3[1],Orbit3[2][150+i,:])
plot(Orbit4[1],Orbit4[2][i,:])
plot!(Orbit4[1],Orbit4[2][50+i,:])
plot!(Orbit4[1],Orbit4[2][100+i,:])
plot!(Orbit4[1],Orbit4[2][150+i,:])
plot!(Orbit4[1],Orbit4[2][i+200,:])
plot!(Orbit4[1],Orbit4[2][250+i,:])
plot!(Orbit4[1],Orbit4[2][300+i,:])
plot!(Orbit4[1],Orbit4[2][350+i,:])
plot!(Orbit5[1],Orbit5[2][i,:])
plot!(Orbit5[1],Orbit5[2][i+50,:])
plot!(Orbit5[1],Orbit5[2][i+100,:])
plot!(Orbit5[1],Orbit5[2][i+150,:])
plot!(GeneTimes,GeneEvolution[i+50*33,:,1],linestyle=:dashdot,linewidth=2)
#ylims!(-0.8,-0.6)
#plot!(Orbit2[1],Orbit2[2][i,:])
#plot!(Orbit2[1],Orbit2[2][i+50,:])
#plot!(Orbit1[1],Orbit1[2][i,:])

In [ ]:
Scoring = zeros(2,N)

for x=1:N
    Differentiation = Differentiaton_Detection_Parameter(M,Final_Epigenetic_Factors[x,:,:]) 

    
    Scoring[:,x] = [Differentiation,x]
end
print(Scoring)



In [ ]:
length(Orbit2[2][14,:])

In [ ]:
Indices = sortslices(Scoring,dims=2,rev=true)
AllIndices = deepcopy(Indices[2,:])
push!(Evolutionary_Complexity,Indices[1,1:K])

Indices = Indices[2,1:K]
Indices = Int.(Indices)
#Indices = [1,2,3,4]
pa = plot(Final_Epigenetic_Factors[Indices[1],:,:],lw=1.5, palette = :tab10) #Final plot (palette is written for M=10, adjust manually if needed)
xlabel!("Cell Position")
ylabel!("Epigenetic")
ylims!(-1, 1)

pb = plot(Final_Epigenetic_Factors[Indices[2],:,:],lw=1.5, palette = :tab10) #Final plot (palette is written for M=10, adjust manually if needed)
xlabel!("Cell Position")
ylabel!("Epigenetic")
ylims!(-1, 1)

pc = plot(Final_Epigenetic_Factors[Indices[3],:,:],lw=1.5, palette = :tab10) #Final plot (palette is written for M=10, adjust manually if needed)
xlabel!("Cell Position")
ylabel!("Epigenetic")
ylims!(-1, 1)

pd = plot(Final_Epigenetic_Factors[Indices[4],:,:],lw=1.5, palette = :tab10) #Final plot (palette is written for M=10, adjust manually if needed)
xlabel!("Cell Position")
ylabel!("Epigenetic")
ylims!(-1, 1)

if K==8
    pe = plot(Final_Epigenetic_Factors[Indices[5],:,:],lw=1.5, palette = :tab10) #Final plot (palette is written for M=10, adjust manually if needed)
    xlabel!("Cell Position")
    ylabel!("Epigenetic")
    ylims!(-1, 1)

    pf = plot(Final_Epigenetic_Factors[Indices[6],:,:],lw=1.5, palette = :tab10) #Final plot (palette is written for M=10, adjust manually if needed)
    xlabel!("Cell Position")
    ylabel!("Epigenetic")
    ylims!(-1, 1)

    pg = plot(Final_Epigenetic_Factors[Indices[7],:,:],lw=1.5, palette = :tab10) #Final plot (palette is written for M=10, adjust manually if needed)
    xlabel!("Cell Position")
    ylabel!("Epigenetic")
    ylims!(-1, 1)

    ph = plot(Final_Epigenetic_Factors[Indices[8],:,:],lw=1.5, palette = :tab10) #Final plot (palette is written for M=10, adjust manually if needed)
    xlabel!("Cell Position")
    ylabel!("Epigenetic")
    ylims!(-1, 1)

    k = plot(pa,pb,pc,pd,pe,pf,pg,ph,layout=(4,2),legend=false)
else
    k = plot(pa,pb,pc,pd,layout=(2,2),legend=false)
end

In [ ]:
plot_array = Any[] # can type this more strictly
for i in 1:64
    
  push!(plot_array,plot(Final_Epigenetic_Factors[Int32(AllIndices[i]),:,:])) # make a plot and add it to the plot_array
end
plot!(plot_array...,layout=(8,8),lw=1.5, palette = :tab10,legend = false,size=(1000,1000))


In [ ]:
array = [i for i=1:K]
print(array)

In [ ]:
# We did it once now, now we put it in function form and keep repeating it

In [ ]:
#Defining it all in function form

function Mutating_Network(N,M,K,ParameterMatrix_Collection,PositionArray_Collection, Indices, Counter, Genetics)
    Updated_Genes = zeros(N)
    Mutations = 1
    Seed_Mutations = 1234
    Indexation_Mutate = 1
    Temporary_Matrix = zeros(Float32,N,M,M)
    Temporary_Vector = zeros(Float32,N,2*M)
    for p = 1:K
        for q=1:N/K
            Temporary_Matrix[Indexation_Mutate,:,:] = Mutate_Parameters(ParameterMatrix_Collection[Indices[p],:,:],M,Mutations,Seed_Mutations+Counter)
            Temporary_Vector[Indexation_Mutate,:] = PositionArray_Collection[Indices[p],:]
            Updated_Genes[Indexation_Mutate] = Genetics[p]
            Counter += 1
            Indexation_Mutate +=1
        end
        Temporary_Matrix[Indexation_Mutate-1,:,:] = ParameterMatrix_Collection[Indices[p],:,:] #We overwrite it to add the original back
    end
    ParameterMatrix_Collection = deepcopy(Temporary_Matrix);
    PositionArray_Collection = deepcopy(Temporary_Vector);
    return ParameterMatrix_Collection, PositionArray_Collection, Counter, Updated_Genes
end

function Running_Network(CArray,ParameterMatrix,PositionArray)
    #CArray = CArray_Collection[x,:]
    #ParameterMatrix = ParameterMatrix_Collection[x,:,:]
    #PositionArray = PositionArray_Collection[x,:]
    for s=1:Splits #Looping over the splits
        Parameters = (ParameterMatrix, CArray,KArray,Int32(s-1)) #We have to update the splits every rerun, rest stays same
        tspan = (Float32(0.0),SplitTime) #How long ODE will run
        prob = ODEProblem(Epigenetic_Evolution3,PositionArray,tspan,Parameters) #ODEdefining
        sol = solve(prob,BS3()) #Solving
        Times = sol.t #Save the time, doesn't get used yet though
        StepAmount = length(Times)  #Amount of data points per gene
        Genes = sol 
        PositionArray = Genes[StepAmount] #Saving the genes and epi factors

        #Now we must take all the values from the updated PositionArray and give appropriate terms to the daughter cells
        PositionArray = Noisy_Splitting(PositionArray,s,M,Random_Noise)  #Splitting function

    end

    Final_Run_Time = Float32(5000) #Long evolving of the cells after splitting to get the final epigenetic state
    Parameters = (ParameterMatrix, CArray,KArray,Splits) 
    tspan = (Float32(0.0),Final_Run_Time)

    if Mean_Field
        prob = ODEProblem(Epigenetic_Evolution2,PositionArray,tspan,Parameters) #ODEdefining
    else
        prob = ODEProblem(Epigenetic_Evolution3,PositionArray,tspan,Parameters) #ODEdefining
    end    
    sol = solve(prob,BS3()) #Solving
    Times = sol.t
    StepAmount = length(Times)
    GenesEpi = sol
    PositionArray = GenesEpi[StepAmount]
    Genes = PositionArray[1:2^Splits*M] #Saves all the final genes
    Epigenetic = PositionArray[1+2^Splits*M:2^(Splits+1)*M]; #Saves all the final Epigenetic factors
    
    #FinalOrbit = [Times,GenesEpi[1,1:M*2^5,1:StepAmount]]
    #FinalEpiOrbit = GenesEpi[1,M*2^5+1:M*2^6,1:StepAmount]
    
    Epigenetic_Plot = zeros(Int.(length(Epigenetic)/M),M)
    for l=1:M
        
        Epigenetic_Plot[:,l] = Epigenetic[l:M:end]  #Reshapes the factors to plot easily    
    end
    
    #Final_Epigenetic_Factors[x,:,:] = Epigenetic_Plot
    return Epigenetic_Plot
end

function Scoring_Indexes(N,Final_Epigenetic_Factors,Evolutionary_Complexity,New_Genes)
    Scoring = zeros(3,N)
    for x=1:N
        Differentiation = Differentiaton_Detection_Parameter(M,Final_Epigenetic_Factors[x,:,:]) 

        Scoring[1:2,x] = [Differentiation,x]
    end
    Scoring[3,:] = New_Genes
    Indices = sortslices(Scoring,dims=2,rev=true)
    Genetics = deepcopy(Indices[3,1:K])
    push!(Evolutionary_Complexity,Indices[1,1:K])
    AllIndices = deepcopy(Indices[2,:])
    Indices = Indices[2,1:K]
    Indices = Int.(Indices)
    #print(Indices[2,1:K])
    
    return Indices, AllIndices, Evolutionary_Complexity, Genetics
end

function Epi_Plotter(Indices,K,Final_Epigenetic_Factors)
    
    pa = plot(Final_Epigenetic_Factors[Indices[1],:,:],lw=1.5, palette = :tab10) #Final plot (palette is written for M=10, adjust manually if needed)
    xlabel!("Cell Position")
    ylabel!("Epigenetic")
    ylims!(-1, 1)

    pb = plot(Final_Epigenetic_Factors[Indices[2],:,:],lw=1.5, palette = :tab10) #Final plot (palette is written for M=10, adjust manually if needed)
    xlabel!("Cell Position")
    ylabel!("Epigenetic")
    ylims!(-1, 1)

    pc = plot(Final_Epigenetic_Factors[Indices[3],:,:],lw=1.5, palette = :tab10) #Final plot (palette is written for M=10, adjust manually if needed)
    xlabel!("Cell Position")
    ylabel!("Epigenetic")
    ylims!(-1, 1)

    pd = plot(Final_Epigenetic_Factors[Indices[4],:,:],lw=1.5, palette = :tab10) #Final plot (palette is written for M=10, adjust manually if needed)
    xlabel!("Cell Position")
    ylabel!("Epigenetic")
    ylims!(-1, 1)

    if K==8
        pe = plot(Final_Epigenetic_Factors[Indices[5],:,:],lw=1.5, palette = :tab10) #Final plot (palette is written for M=10, adjust manually if needed)
        xlabel!("Cell Position")
        ylabel!("Epigenetic")
        ylims!(-1, 1)

        pf = plot(Final_Epigenetic_Factors[Indices[6],:,:],lw=1.5, palette = :tab10) #Final plot (palette is written for M=10, adjust manually if needed)
        xlabel!("Cell Position")
        ylabel!("Epigenetic")
        ylims!(-1, 1)

        pg = plot(Final_Epigenetic_Factors[Indices[7],:,:],lw=1.5, palette = :tab10) #Final plot (palette is written for M=10, adjust manually if needed)
        xlabel!("Cell Position")
        ylabel!("Epigenetic")
        ylims!(-1, 1)

        ph = plot(Final_Epigenetic_Factors[Indices[8],:,:],lw=1.5, palette = :tab10) #Final plot (palette is written for M=10, adjust manually if needed)
        xlabel!("Cell Position")
        ylabel!("Epigenetic")
        ylims!(-1, 1)

        k = plot(pa,pb,pc,pd,pe,pf,pg,ph,layout=(4,2),legend=false)
    else
        k = plot(pa,pb,pc,pd,layout=(2,2),legend=false)
    end
    return k
end

In [ ]:
Runs = [4, 4, 4, 4,4,4]
#Runs .= 1

#Short version of it all
Genetics = [i for i=1:K] #Generate our first generation of genetic trackers
Genetic_History = 
ParameterMatrix_Collection, PositionArray_Collection, Counter, New_Genes = Mutating_Network(N,M,K,ParameterMatrix_Collection,PositionArray_Collection, Indices, Counter,Genetics)
Genetic_History = Genetics
@showprogress 1 for x=1:N
    #CArray = CArray_Collection[x,:]
    #ParameterMatrix = ParameterMatrix_Collection[x,:,:]
    #PositionArray = PositionArray_Collection[x,:]
    Final_Epigenetic_Factors[x,:,:] = Running_Network(CArray_Collection[x,:],ParameterMatrix_Collection[x,:,:],PositionArray_Collection[x,:])
    
end

Indices,AllIndices,Evolutionary_Complexity, Genetics = Scoring_Indexes(N,Final_Epigenetic_Factors,Evolutionary_Complexity,New_Genes)
Genetic_History = hcat(Genetic_History,Genetics)
Figure = Epi_Plotter(Indices,K,Final_Epigenetic_Factors)
display(Figure)
sleep(0.1)
plot_array = Any[] # can type this more strictly
for i in 1:64
    push!(plot_array,plot(Final_Epigenetic_Factors[Int32(AllIndices[i]),:,:])) # make a plot and add it to the plot_array
end
plot!(plot_array...,layout=(8,8),lw=1.5, palette = :tab10,legend = false,size=(1000,1000))

In [ ]:

print(Genetic_History[:,2])
print(mean.(Evolutionary_Complexity[:,1]))
plot(mean.(Evolutionary_Complexity[:,1]))

In [ ]:
@showprogress 1 for q=1:Runs[1]
    #Short version of it all
    ParameterMatrix_Collection, PositionArray_Collection, Counter, New_Genes = Mutating_Network(N,M,K,ParameterMatrix_Collection,PositionArray_Collection, Indices, Counter,Genetics)
    @showprogress 2 for x=1:N
        #CArray = CArray_Collection[x,:]
        #ParameterMatrix = ParameterMatrix_Collection[x,:,:]
        #PositionArray = PositionArray_Collection[x,:]
        Final_Epigenetic_Factors[x,:,:] = Running_Network(CArray_Collection[x,:],ParameterMatrix_Collection[x,:,:],PositionArray_Collection[x,:])

    end

    Indices,AllIndices,Evolutionary_Complexity, Genetics = Scoring_Indexes(N,Final_Epigenetic_Factors,Evolutionary_Complexity,New_Genes)
    Genetic_History = hcat(Genetic_History,Genetics)
    Figure = Epi_Plotter(Indices,K,Final_Epigenetic_Factors)
    display(Figure)
    sleep(0.1)
    plot_array = Any[] # can type this more strictly
    for i in 1:64
    
        push!(plot_array,plot(Final_Epigenetic_Factors[Int32(AllIndices[i]),:,:])) # make a plot and add it to the plot_array
    end
    plot!(plot_array...,layout=(8,8),lw=1.5, palette = :tab10,legend = false,size=(1000,1000))
    sleep(0.1)
end

In [ ]:
print(Evolutionary_Complexity)

In [ ]:
print(mean.(Evolutionary_Complexity[:,1]))
plot(mean.(Evolutionary_Complexity[:,1]))

In [ ]:
@showprogress 1 for q=1:Runs[2]
    #Short version of it all
    ParameterMatrix_Collection, PositionArray_Collection, Counter, New_Genes = Mutating_Network(N,M,K,ParameterMatrix_Collection,PositionArray_Collection, Indices, Counter,Genetics)
    @showprogress 2 for x=1:N
        #CArray = CArray_Collection[x,:]
        #ParameterMatrix = ParameterMatrix_Collection[x,:,:]
        #PositionArray = PositionArray_Collection[x,:]
        Final_Epigenetic_Factors[x,:,:] = Running_Network(CArray_Collection[x,:],ParameterMatrix_Collection[x,:,:],PositionArray_Collection[x,:])

    end

    Indices,AllIndices,Evolutionary_Complexity, Genetics = Scoring_Indexes(N,Final_Epigenetic_Factors,Evolutionary_Complexity,New_Genes)
    Genetic_History = hcat(Genetic_History,Genetics)
    Figure = Epi_Plotter(Indices,K,Final_Epigenetic_Factors)
    display(Figure)
    sleep(0.1)
    plot_array = Any[] # can type this more strictly
    for i in 1:64
    
        push!(plot_array,plot(Final_Epigenetic_Factors[Int32(AllIndices[i]),:,:])) # make a plot and add it to the plot_array
    end
    plot!(plot_array...,layout=(8,8),lw=1.5, palette = :tab10,legend = false,size=(1000,1000))
    sleep(0.1)
end

In [ ]:
print(Genetic_History)

In [ ]:
print(Evolutionary_Complexity)
print(mean.(Evolutionary_Complexity[:,1]))
plot(mean.(Evolutionary_Complexity[:,1]))

In [ ]:
@showprogress 1 for q=1:Runs[3]
    #Short version of it all
    ParameterMatrix_Collection, PositionArray_Collection, Counter, New_Genes = Mutating_Network(N,M,K,ParameterMatrix_Collection,PositionArray_Collection, Indices, Counter,Genetics)
    @showprogress 2 for x=1:N
        #CArray = CArray_Collection[x,:]
        #ParameterMatrix = ParameterMatrix_Collection[x,:,:]
        #PositionArray = PositionArray_Collection[x,:]
        Final_Epigenetic_Factors[x,:,:] = Running_Network(CArray_Collection[x,:],ParameterMatrix_Collection[x,:,:],PositionArray_Collection[x,:])

    end

    Indices,AllIndices,Evolutionary_Complexity, Genetics = Scoring_Indexes(N,Final_Epigenetic_Factors,Evolutionary_Complexity,New_Genes)
    Genetic_History = hcat(Genetic_History,Genetics)
    Figure = Epi_Plotter(Indices,K,Final_Epigenetic_Factors)
    display(Figure)
    sleep(0.1)
    plot_array = Any[] # can type this more strictly
    for i in 1:64
    
        push!(plot_array,plot(Final_Epigenetic_Factors[Int32(AllIndices[i]),:,:])) # make a plot and add it to the plot_array
    end
    plot!(plot_array...,layout=(8,8),lw=1.5, palette = :tab10,legend = false,size=(1000,1000))
    sleep(0.1)
end

In [ ]:
print(Evolutionary_Complexity)
print(mean.(Evolutionary_Complexity[:,1]))
plot(mean.(Evolutionary_Complexity[:,1]))

In [ ]:
@showprogress 1 for q=1:Runs[4]
    #Short version of it all
    ParameterMatrix_Collection, PositionArray_Collection, Counter, New_Genes = Mutating_Network(N,M,K,ParameterMatrix_Collection,PositionArray_Collection, Indices, Counter,Genetics)
    @showprogress 2 for x=1:N
        #CArray = CArray_Collection[x,:]
        #ParameterMatrix = ParameterMatrix_Collection[x,:,:]
        #PositionArray = PositionArray_Collection[x,:]
        Final_Epigenetic_Factors[x,:,:] = Running_Network(CArray_Collection[x,:],ParameterMatrix_Collection[x,:,:],PositionArray_Collection[x,:])

    end

    Indices,AllIndices,Evolutionary_Complexity, Genetics = Scoring_Indexes(N,Final_Epigenetic_Factors,Evolutionary_Complexity,New_Genes)
    Genetic_History = hcat(Genetic_History,Genetics)
    Figure = Epi_Plotter(Indices,K,Final_Epigenetic_Factors)
    display(Figure)
    sleep(0.1)
    plot_array = Any[] # can type this more strictly
    for i in 1:64
    
        push!(plot_array,plot(Final_Epigenetic_Factors[Int32(AllIndices[i]),:,:])) # make a plot and add it to the plot_array
    end
    plot!(plot_array...,layout=(8,8),lw=1.5, palette = :tab10,legend = false,size=(1000,1000))
    sleep(0.1)
end

In [ ]:
print(Evolutionary_Complexity)
print(mean.(Evolutionary_Complexity[:,1]))
plot(mean.(Evolutionary_Complexity[:,1]))

In [ ]:
@showprogress 1 for q=1:Runs[5]
    #Short version of it all
    ParameterMatrix_Collection, PositionArray_Collection, Counter, New_Genes = Mutating_Network(N,M,K,ParameterMatrix_Collection,PositionArray_Collection, Indices, Counter,Genetics)
    @showprogress 2 for x=1:N
        #CArray = CArray_Collection[x,:]
        #ParameterMatrix = ParameterMatrix_Collection[x,:,:]
        #PositionArray = PositionArray_Collection[x,:]
        Final_Epigenetic_Factors[x,:,:] = Running_Network(CArray_Collection[x,:],ParameterMatrix_Collection[x,:,:],PositionArray_Collection[x,:])

    end

    Indices,AllIndices,Evolutionary_Complexity, Genetics = Scoring_Indexes(N,Final_Epigenetic_Factors,Evolutionary_Complexity,New_Genes)
    Genetic_History = hcat(Genetic_History,Genetics)
    Figure = Epi_Plotter(Indices,K,Final_Epigenetic_Factors)
    display(Figure)
    sleep(0.1)
    plot_array = Any[] # can type this more strictly
    for i in 1:64
    
        push!(plot_array,plot(Final_Epigenetic_Factors[Int32(AllIndices[i]),:,:])) # make a plot and add it to the plot_array
    end
    plot!(plot_array...,layout=(8,8),lw=1.5, palette = :tab10,legend = false,size=(1000,1000))
    sleep(0.1)
end

In [ ]:
print(Evolutionary_Complexity)
print(mean.(Evolutionary_Complexity[:,1]))
plot(mean.(Evolutionary_Complexity[:,1]))

In [ ]:
@showprogress 1 for q=1:Runs[6]
    #Short version of it all
    ParameterMatrix_Collection, PositionArray_Collection, Counter, New_Genes = Mutating_Network(N,M,K,ParameterMatrix_Collection,PositionArray_Collection, Indices, Counter,Genetics)
    @showprogress 2 for x=1:N
        #CArray = CArray_Collection[x,:]
        #ParameterMatrix = ParameterMatrix_Collection[x,:,:]
        #PositionArray = PositionArray_Collection[x,:]
        Final_Epigenetic_Factors[x,:,:] = Running_Network(CArray_Collection[x,:],ParameterMatrix_Collection[x,:,:],PositionArray_Collection[x,:])

    end

    Indices,AllIndices,Evolutionary_Complexity, Genetics = Scoring_Indexes(N,Final_Epigenetic_Factors,Evolutionary_Complexity,New_Genes)
    Genetic_History = hcat(Genetic_History,Genetics)
    Figure = Epi_Plotter(Indices,K,Final_Epigenetic_Factors)
    display(Figure)
    sleep(0.1)
    plot_array = Any[] # can type this more strictly
    for i in 1:64
    
        push!(plot_array,plot(Final_Epigenetic_Factors[Int32(AllIndices[i]),:,:])) # make a plot and add it to the plot_array
    end
    plot!(plot_array...,layout=(8,8),lw=1.5, palette = :tab10,legend = false,size=(1000,1000))
    sleep(0.1)
end

In [ ]:
print(Evolutionary_Complexity)
print(mean.(Evolutionary_Complexity[:,1]))
plot(mean.(Evolutionary_Complexity[:,1]))

In [ ]:
plot(mean.(Evolutionary_Complexity[:,1]))

In [ ]:
print(Genetic_History)
#Make a loop for all different generations
#In this loop we iterate over Genetic_History[:,i], we add counters to the respective family and then go i+1
#Once we went over it all we divide by length(Genetic_History[:,1]), then we do the following:
#Family 1 = Family 1
#Family 2 += Family 1
#Family 3 += Family 2 and so on, this should give cumulative values
#Check final family to end up at 1
#Once finished we make a loop that fills in the ranges between and we get something like below

In [ ]:
Gener = length(Genetic_History[1,:]) #How many generations were there
Fam = length(Genetic_History[:,1]) #How many families to start
Family_Evolve = zeros(Gener,Fam)
for g=1:Gener
    for f=1:Fam
        Family_Evolve[g,Int.(Genetic_History[f,g])] += 1 
    end
end

print(Family_Evolve)

for f=1:Fam-1
    Family_Evolve[:,f+1] += Family_Evolve[:,f] 
end

print(Family_Evolve)
print(Family_Evolve[1:3,Fam])

In [ ]:
using Plots
a = [1,3,7,10]
b = [1,5,8,17]
plot(a,fillrange=[zeros(4) a])
plot!(b,fillrange=[a b])
plot!(fillrange=[a b])


In [ ]:
@showprogress 1 for q=1:100
    #Short version of it all
    ParameterMatrix_Collection, PositionArray_Collection, Counter, New_Genes = Mutating_Network(N,M,K,ParameterMatrix_Collection,PositionArray_Collection, Indices, Counter,Genetics)
    for x=1:N
        #CArray = CArray_Collection[x,:]
        #ParameterMatrix = ParameterMatrix_Collection[x,:,:]
        #PositionArray = PositionArray_Collection[x,:]
        Final_Epigenetic_Factors[x,:,:] = Running_Network(CArray_Collection[x,:],ParameterMatrix_Collection[x,:,:],PositionArray_Collection[x,:])

    end

    Indices,AllIndices,Evolutionary_Complexity, Genetics = Scoring_Indexes(N,Final_Epigenetic_Factors,Evolutionary_Complexity,New_Genes)
    Genetic_History = hcat(Genetic_History,Genetics)
    Figure = Epi_Plotter(Indices,K,Final_Epigenetic_Factors)
    display(Figure)
    sleep(0.1)
    plot_array = Any[] # can type this more strictly
    for i in 1:64
    
        push!(plot_array,plot(Final_Epigenetic_Factors[Int32(AllIndices[i]),:,:])) # make a plot and add it to the plot_array
    end
    plot!(plot_array...,layout=(8,8),lw=1.5, palette = :tab10,legend = false,size=(1000,1000))
    sleep(0.1)
end

In [ ]:
print(Evolutionary_Complexity)
print(mean.(Evolutionary_Complexity[:,1]))
plot(mean.(Evolutionary_Complexity[:,1]))

In [ ]:
@showprogress 1 for q=1:40
    #Short version of it all
    ParameterMatrix_Collection, PositionArray_Collection, Counter, New_Genes = Mutating_Network(N,M,K,ParameterMatrix_Collection,PositionArray_Collection, Indices, Counter,Genetics)
    for x=1:N
        #CArray = CArray_Collection[x,:]
        #ParameterMatrix = ParameterMatrix_Collection[x,:,:]
        #PositionArray = PositionArray_Collection[x,:]
        Final_Epigenetic_Factors[x,:,:] = Running_Network(CArray_Collection[x,:],ParameterMatrix_Collection[x,:,:],PositionArray_Collection[x,:])

    end

    Indices,AllIndices,Evolutionary_Complexity, Genetics = Scoring_Indexes(N,Final_Epigenetic_Factors,Evolutionary_Complexity,New_Genes)
    Genetic_History = hcat(Genetic_History,Genetics)
    Figure = Epi_Plotter(Indices,K,Final_Epigenetic_Factors)
    display(Figure)
    sleep(0.1)
    plot_array = Any[] # can type this more strictly
    for i in 1:64
    
        push!(plot_array,plot(Final_Epigenetic_Factors[Int32(AllIndices[i]),:,:])) # make a plot and add it to the plot_array
    end
    plot!(plot_array...,layout=(8,8),lw=1.5, palette = :tab10,legend = false,size=(1000,1000))
    sleep(0.1)
end

In [ ]:
print(Evolutionary_Complexity)
print(mean.(Evolutionary_Complexity[:,1]))
plot(mean.(Evolutionary_Complexity[:,1]))